In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
import lib

# Set random seed for reproducibility
random_state = 0

#### Config

In [3]:
### LLM Model

## OpenAI
# https://platform.openai.com/docs/models
CLIENT_TYPE = lib.agent.ClientType.OPENAI
MODEL_NAME = 'gpt-4o-mini'
# MODEL_NAME = 'gpt-4.1-nano'
# MODEL_NAME = 'gpt-4.1-mini'
# MODEL_NAME = 'gpt-4.1'

## Groq
# CLIENT_TYPE = lib.agent.ClientType.GROQ
# MODEL_NAME = 'llama-3.1-8b-instant'
# MODEL_NAME = 'llama-3.3-70b-versatile'

## TogetherAI 
# CLIENT_TYPE = lib.agent.ClientType.TOGETHER
# MODEL_NAME = 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo'

### Dataset source

SOURCE = 'ToolE'
# SOURCE = 'Berkeley'
# SOURCE = 'ToolLens'
# SOURCE = 'ReverseChain'

### Agent setup

# setupType = lib.experiment.ExperimentSetupType.REACT_SINGLE_STEP
# setupType = lib.experiment.ExperimentSetupType.REACT
# setupType = lib.experiment.ExperimentSetupType.REACT_WITH_CO_OCCURRENCE
# setupType = lib.experiment.ExperimentSetupType.QUOTA
setupType = lib.experiment.ExperimentSetupType.QUOTA_WITH_REFLECTION

(react_agent, _, _, _, _, prompt_single, prompt_multi, _, _, _) = lib.experiment.load_experiment_setup(
    setupType,
    CLIENT_TYPE,
    MODEL_NAME,
    SOURCE,
    2000,
    random_state=random_state,
)

Data source: ToolE, 2000 single tool samples, 497 multi tool samples.
Initializing reflective action queue...
Initializing ReAct agent...
Experiment setup: Quota (Quota with Reflection)
	Model: gpt-4o-mini
	Temperature: 0
	Max Turns: 15
	Top-K Tools: 5
	Similarity Threshold: 0.8
	Action Queue: True
	Co-occurrence: True


#### Test run

In [4]:
QUERY = "Show me the current price of Bitcoin in USD."
Y_TOOLS = ['tool_e.finance_tool']

# QUERY = "I want to know the latest news about Tesla and how it has impacted the stock market."
# Y_TOOLS = ['tool_e.finance_tool', 'tool_e.news_tool']

Y_TOOLS = []
res_text, res_state = await react_agent.invoke(
    QUERY,
    system_prompt=prompt_single if len(Y_TOOLS) == 1 else prompt_multi,
    true_tools=Y_TOOLS,
    verbose=True,
    )


[19:13:33] ============================================== Prompt ==============================================

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer

## Instructions
1. **Thought** - Describe your thoughts about the question you have been asked and create a plan of action.
2. **Action** - Run one of the actions available to you, then return PAUSE.
3. **PAUSE** - Wait for the observation to be returned.
4. **Observation** - This will be the result of running those actions.
5. Two choices:
    - **Answer** - If you have enough information to answer the question, output your answer
    - **Repeat from Thought step** - If you need to run another action, repeat from the Thought step.

Your actions are:

find_action:
e.g. find_action: { "query": "Description of the desired tool" }
Returns a ranked list of external actions that semantically match the given description, based on sentence similarity. The action is idempotent: re

### Agent state

In [5]:
print(res_state.model_dump_json(indent=2))

{
  "question": "Show me the current price of Bitcoin in USD.",
  "messages": [
    {
      "role": "user",
      "content": "Show me the current price of Bitcoin in USD."
    },
    {
      "role": "assistant",
      "content": "Thought: I need to find an action that can provide the current price of Bitcoin in USD. This will likely involve querying a financial or cryptocurrency data source. \nAction: find_action: { \"query\": \"current price of Bitcoin in USD\" }\nPAUSE"
    },
    {
      "role": "user",
      "content": "observation 1: Found the following actions, which I can use:\n\ntool_e.exchange_tool:\ne.g. tool_e.exchange_tool: {\"action_input\": \"input\"}\nSeamlessly convert currencies with our integrated currency conversion tool. This tool cannot be used for translating languages or engaging in conversations with AI tutors, as it solely focuses on converting currencies. For that purpose, you can also use the tool 'tool_e.mixer_box__translate__a_i_language_tutor' instead if i